# 🤗 `transformers`

More info about using `transformers` models with outlines [here](https://dottxt-ai.github.io/outlines/latest/reference/models/transformers/)

Table of Contents:
- [JSON Generation](#json-generation)
- [Text Generation](#text-generation)


## JSON Generation

Example based on https://dottxt-ai.github.io/outlines/latest/cookbook/extraction/

In [1]:
from enum import Enum

import jinja2
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from pydantic import BaseModel

from outlines_haystack.generators.transformers import TransformersJSONGenerator

In [2]:
class Pizza(str, Enum):
    margherita = "Margherita"
    calzone = "Calzone"


class Order(BaseModel):
    pizza: Pizza
    number: int

In [3]:
prompt_template = """You are the owner of a pizza parlor. Customers \
send you orders from which you need to extract:

1. The pizza that is ordered
2. The number of pizzas

# EXAMPLE

ORDER: I would like one Margherita pizza
RESULT: {"pizza": "Margherita", "number": 1}

# OUTPUT INSTRUCTIONS

Answer in valid JSON. Here are the different objects relevant for the output:

Order:
    pizza (str): name of the pizza
    number (int): number of pizzas

Return a valid JSON of type "Order"

# OUTPUT

ORDER: {{ order }}
RESULT: """

In [4]:
generator = TransformersJSONGenerator(
    model_name="microsoft/Phi-3-mini-4k-instruct",
    schema_object=Order,
    sampling_algorithm_kwargs={"temperature": 0.5},
)

### Standalone

In [5]:
generator.warm_up()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# we use Jinja2 to render the template
prompt = jinja2.Template(prompt_template).render(order="Is it possible to have 12 margheritas?")
generator.run(prompt=prompt)

{'structured_replies': [{'pizza': 'Margherita', 'number': 12}]}

### In a Pipeline

In [7]:
pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=TransformersJSONGenerator(
        model_name="microsoft/Phi-3-mini-4k-instruct",
        schema_object=Order,
        sampling_algorithm_kwargs={"temperature": 0.5},
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: TransformersJSONGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [8]:
pipeline.run({"prompt_builder": {"order": "Is it possible to have 12 margheritas?"}})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

{'llm': {'structured_replies': [{'pizza': 'Margherita', 'number': 12}]}}

## Choice Generation

In [1]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.transformers import TransformersChoiceGenerator

In [2]:
generator = TransformersChoiceGenerator(
    model_name="microsoft/Phi-3-mini-4k-instruct",
    choices=["Positive", "Negative"],
    sampling_algorithm_kwargs={"temperature": 0.5},
)

### Standalone

In [3]:
generator.warm_up()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
generator.run(prompt="Classify the following statement: 'I love pizza'")

{'choice': 'Positive'}

### In a Pipeline

In [5]:
prompt_template = "Classify the following statement: '{{statement}}'"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=TransformersChoiceGenerator(
        model_name="microsoft/Phi-3-mini-4k-instruct",
        choices=["Positive", "Negative"],
        sampling_algorithm_kwargs={"temperature": 0.5},
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: TransformersChoiceGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [6]:
pipeline.run({"prompt_builder": {"statement": "I love Italian food"}})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

{'llm': {'choice': 'Positive'}}

## Text Generation

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

In [2]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.transformers import TransformersTextGenerator

In [3]:
generator = TransformersTextGenerator(
    model_name="microsoft/Phi-3-mini-4k-instruct",
    device=device,
    sampling_algorithm_kwargs={"temperature": 0.5},
)

### Standalone

In [4]:
generator.warm_up()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
generator.run(prompt="What is the capital of Italy?")

{'replies': ['\n\n# Answer\nThe capital of Italy is Rome.']}

### In a Pipeline

In [6]:
prompt_template = "What is the capital of {{country}}?"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=TransformersTextGenerator(
        model_name="microsoft/Phi-3-mini-4k-instruct",
        device=device,
        sampling_algorithm_kwargs={"temperature": 0.5},
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: TransformersTextGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [7]:
pipeline.run({"prompt_builder": {"country": "France"}})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

{'llm': {'replies': ['\n\n# Answer\nThe capital of France is Paris.']}}